In [1]:
from algosdk import kmd, mnemonic,account
from algosdk.wallet import Wallet
from algosdk.future.transaction import *
from algosdk.encoding import decode_address,encode_address
from algosdk.v2client import algod
from algosdk.future import transaction
from pyteal import compileTeal, Mode
import algosdk.encoding as enc

import json
import base64


In [2]:
# create a kmd client
kmd_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"
kmd_address = "http://94.74.95.107:4002"
kcl = kmd.KMDClient(kmd_token, kmd_address)

# link the wallet
default_wallet_name="unencrypted-default-wallet"
default_wallet_sec=""
default_wallet = Wallet(default_wallet_name, default_wallet_sec, kcl)
info = default_wallet.info()
print("Wallet name:", info["wallet"]["name"])

walletid = None
wallets = kcl.list_wallets()
for arrayitem in wallets:
    if arrayitem.get("name") == default_wallet_name:
        walletid = arrayitem.get("id")
        break
default_wallethandle = kcl.init_wallet_handle(walletid, default_wallet_sec)

sponsor = "KI5HOKIYGKEOZ6VLNHKJGF4KXYNBZHXB7HSX5BQ2LYNB2IJLWFT7DIV2XI"
sponsorkey = kcl.export_key(default_wallethandle, default_wallet_sec, sponsor)
sponsormn = mnemonic.from_private_key(sponsorkey)
print(f"\nAccount: {sponsor}\nMnemonic: {sponsormn}\nKey: {sponsorkey}")

# link the wallet
wallet_name="Metaone"
wallet_sec=""
wallet = Wallet(wallet_name, wallet_sec, kcl)
info = wallet.info()
print("\nWallet name:", info["wallet"]["name"])

walletid = None
wallets = kcl.list_wallets()
for arrayitem in wallets:
    if arrayitem.get("name") == wallet_name:
        walletid = arrayitem.get("id")
        break
wallethandle = kcl.init_wallet_handle(walletid, wallet_sec)

# create accounts
for i in range(0,3):
    address = wallet.generate_key()
#     print("New account:", address)
    
accounts = wallet.list_keys()
# print("accounts:", accounts)

accountkey1 = kcl.export_key(wallethandle, wallet_sec, accounts[0])
mn1 = mnemonic.from_private_key(accountkey1)
print(f"\nAccount: {accounts[0]}\nMnemonic: {mn1}\nKey: {accountkey1}")

accountkey2 = kcl.export_key(wallethandle, wallet_sec, accounts[1])
mn2 = mnemonic.from_private_key(accountkey2)
print(f"\nAccount: {accounts[1]}\nMnemonic: {mn2}\nKey: {accountkey2}")

accountkey3 = kcl.export_key(wallethandle, wallet_sec, accounts[2])
mn3 = mnemonic.from_private_key(accountkey3)
print(f"\nAccount: {accounts[2]}\nMnemonic: {mn3}\nKey: {accountkey3}")


#create multisig account for management
version = 1  # multisig version
threshold = 2  # how many signatures are necessary
msig = Multisig(version, threshold, [accounts[0], accounts[1], accounts[2]])
print("\nMultisig account",msig.address())

msig.validate()
wallet.import_multisig(msig)
print("\nMultisig account list",wallet.list_multisig())

Wallet name: unencrypted-default-wallet

Account: KI5HOKIYGKEOZ6VLNHKJGF4KXYNBZHXB7HSX5BQ2LYNB2IJLWFT7DIV2XI
Mnemonic: luxury shuffle curious keep fashion power direct fruit segment chest napkin decide infant pepper glide two stage found solar tooth snap garage depth abstract cupboard
Key: Ktzxa5ynKaXSx10Z5skljaO5i2Ks66EO15xNruZ9ZedSOncpGDKI7PqradSTF4q+Ghye4fnlfoYaXhodISuxZw==

Wallet name: Metaone

Account: IC3NFRBAJXAWUTIMRUWE5XRFDDZZHX255ZGF7TOK36FLUWZE6I7QDX4KUU
Mnemonic: praise weekend quarter course tiger brief soda father below view warfare project erode add ice trade mushroom level victory razor fee oval wreck absent judge
Key: TEX+XhXz8G/EuVOm+DzuwUqmDASu5o4sYOcrS6p2xr9AttLEIE3Bak0MjSxO3iUY85PfXe5MX83K34q6WyTyPw==

Account: OBAA4DRCRRWSG2AUSJXXY6SPC6Q4VOM3NAW55FCYM7CWTJSBUHG67DNSVU
Mnemonic: common echo afford police awake proud galaxy loop buffalo coast unique invest subject rib crystal lend flag arch point quick bullet road culture absent panther
Key: c3nRCHoaCLPe64PtMAvbX/f

In [3]:
# create a algo client
algod_address = "http://94.74.95.107:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"
algod_client = algod.AlgodClient(algod_token, algod_address)

In [4]:

def pay_start_fund(algod_client, sponsor, sponsorkey, reciever, amount):
    # get node suggested parameters
    params = algod_client.suggested_params()


    # create transaction
    txn = transaction.PaymentTxn(
        sponsor,
        params,
        reciever,
        amount,
    )

    # sign transaction with account
    signed_txn = txn.sign(sponsorkey)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    algod_client.send_transactions([signed_txn])
    print(signed_txn)

    # await confirmation

    confirmed_txn = wait_for_confirmation(algod_client, tx_id, 4)
    print("TX:",txn)
    print("TXID: ", tx_id)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))



In [5]:
fund_amount=100000000
pay_start_fund(algod_client, sponsor, sponsorkey, accounts[0], fund_amount)
pay_start_fund(algod_client, sponsor, sponsorkey, accounts[1], fund_amount)
pay_start_fund(algod_client, sponsor, sponsorkey, accounts[2], fund_amount)
pay_start_fund(algod_client, sponsor, sponsorkey, msig.address(), fund_amount)

TX: {'sender': 'KI5HOKIYGKEOZ6VLNHKJGF4KXYNBZHXB7HSX5BQ2LYNB2IJLWFT7DIV2XI', 'fee': 1000, 'first_valid_round': 16, 'last_valid_round': 1016, 'note': None, 'genesis_id': 'sandnet-v1', 'genesis_hash': 'WhNxQp0oM+XnShfmGyVTIm8yZmpzExyEA0zWyMDWkk0=', 'group': None, 'lease': None, 'type': 'pay', 'rekey_to': None, 'receiver': 'IC3NFRBAJXAWUTIMRUWE5XRFDDZZHX255ZGF7TOK36FLUWZE6I7QDX4KUU', 'amt': 100000000, 'close_remainder_to': None}
TXID:  XN6D2TOWRWTU33RPIHCJKPCIQYIRHSAKJDCVAX5X5BODFJCLBZUQ
Result confirmed in round: 18
TX: {'sender': 'KI5HOKIYGKEOZ6VLNHKJGF4KXYNBZHXB7HSX5BQ2LYNB2IJLWFT7DIV2XI', 'fee': 1000, 'first_valid_round': 18, 'last_valid_round': 1018, 'note': None, 'genesis_id': 'sandnet-v1', 'genesis_hash': 'WhNxQp0oM+XnShfmGyVTIm8yZmpzExyEA0zWyMDWkk0=', 'group': None, 'lease': None, 'type': 'pay', 'rekey_to': None, 'receiver': 'OBAA4DRCRRWSG2AUSJXXY6SPC6Q4VOM3NAW55FCYM7CWTJSBUHG67DNSVU', 'amt': 100000000, 'close_remainder_to': None}
TXID:  4W52YSEQYVRIUGELTZ24KNA6RX3BJPWQTZGRRFR7K7R

In [6]:
# create new application
def create_app(
    client,
    private_key,
    approval_program,
    clear_program,
    global_schema,
    local_schema,
    app_args=None,
    accounts=None,
    
):
    # define sender as creator
    sender = account.address_from_private_key(private_key)

    # declare on_complete as NoOp
    on_complete = transaction.OnComplete.NoOpOC.real

    # get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 1000

    # create unsigned transaction
    txn = transaction.ApplicationCreateTxn(
        sender,
        params,
        on_complete,
        approval_program,
        clear_program,
        global_schema,
        local_schema,
        app_args,
        accounts,
    )

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # await confirmation
    wait_for_confirmation(client, tx_id)

    # display results
    transaction_response = client.pending_transaction_info(tx_id)
    app_id = transaction_response["application-index"]
    print("Created new app-id:", app_id)

    return app_id

def compile_program(client, source_code):
    compile_response = client.compile(source_code)
    return base64.b64decode(compile_response["result"])

In [7]:
# call application
def call_app(client, private_key, index, app_args, another_account=None) : 
    # declare sender
    sender = account.address_from_private_key(private_key)
    print("Call from account: ",sender)

	# get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    # params.flat_fee = True
    # params.fee = 1000


    
    if another_account!=None:
        # create unsigned transaction
        txn = ApplicationNoOpTxn(sender, params, index, app_args, accounts=[another_account])
    else:
        txn = ApplicationNoOpTxn(sender, params, index, app_args)

    # sign transaction

#     if another_key!=None:
#         signed_txn = txn.sign(another_key)
    
    signed_txn = txn.sign(private_key)
    
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])
    
    print(signed_txn)

    # await confirmation

    confirmed_txn = wait_for_confirmation(client, tx_id, 4)
    print("TX:",txn)
    print("TXID: ", tx_id)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))
    # display results
    transaction_response = client.pending_transaction_info(tx_id)
    print("Called app-id: ",transaction_response['txn']['txn']['apid'])
    if "global-state-delta" in transaction_response :
        print("Global State updated :\n")
        print(json.dumps(transaction_response['global-state-delta'], indent=2))
    if "local-state-delta" in transaction_response :
        print("Local State updated :\n")
        print(json.dumps(transaction_response['local-state-delta'], indent=2))
        
        
# read user local state
def read_local_state(client, addr, app_id):
    results = client.account_info(addr)
    local_state = results["apps-local-state"][0]
    for index in local_state:
        if local_state[index] == app_id:
            print(f"local_state of account {addr} for app_id {app_id}: ")


# read app global state
def read_global_state(client, addr, app_id):
    results = client.account_info(addr)
    apps_created = results["created-apps"]
    for app in apps_created:
        if app["id"] == app_id:
#             print(f"global_state for app_id {app_id}: ")
#             print(json.dumps(app["params"]["global-state"], indent=2))
            return app

In [8]:
# mint test tokens for payment
# sp = algod_client.suggested_params()
# txn = AssetConfigTxn(
#     sender=accounts[0],
#     sp=sp,
#     total=1000,
#     default_frozen=False,
#     unit_name="MTO",
#     asset_name="metaone",
#     manager=accounts[0],
#     reserve=accounts[0],
#     freeze=accounts[0],
#     clawback=accounts[0],
#     url="https://path/to/my/asset/details", 
#     decimals=0)

# stxn = txn.sign(accountkey1)

# try:
#     txid = algod_client.send_transaction(stxn)
#     print("Signed transaction with txID: {}".format(txid))
#     # Wait for the transaction to be confirmed
#     confirmed_txn = wait_for_confirmation(algod_client, txid, 4)  
#     print("TXID: ", txid)
#     print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))   
# except Exception as err:
#     print(err)
    
    
    
version = 1  # multisig version
threshold = 2  # how many signatures are necessary
msig = Multisig(version, threshold, [accounts[0], accounts[1], accounts[2]])
msig.validate()

# get node suggested parameters
params = algod_client.suggested_params()
# comment out the next two (2) lines to use suggested fees
# params.flat_fee = True
# params.fee = 1000

txn = AssetConfigTxn(
    sender=msig.address(),
    sp=params,
    total=1000,
    default_frozen=False,
    unit_name="MTO",
    asset_name="metaone",
    manager=msig.address(),
    reserve=msig.address(),
    freeze=msig.address(),
    clawback=msig.address(),
    url="https://path/to/my/asset/details", 
    decimals=0)


mtx = MultisigTransaction(txn, msig)
mtx.sign(accountkey1)
mtx.sign(accountkey2)
# mtx.sign(accountkey3)

# print(json.dumps(mtx.multisig.json_dictify(), indent=2))
# print(mtx.transaction.dictify())
# print("Encoded transaction:", encoding.msgpack_encode(mtx))


try:
# send the transaction
    txid = algod_client.send_raw_transaction(encoding.msgpack_encode(mtx))    
    print("TXID: ", txid)   
    confirmed_txn = wait_for_confirmation(algod_client, txid, 6)  
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))
    print("Transaction information: {}".format(
        json.dumps(confirmed_txn, indent=4)))
    print("Decoded note: {}".format(base64.b64decode(
        confirmed_txn["txn"]["txn"]["note"]).decode()))
except Exception as err:
    print(err)    
    
account_info = algod_client.account_info(msig.address())
print("balance of {}: {} microAlgos".format(msig.address, account_info["amount"]))

holding_assets = account_info["assets"]
print(f"holding assets of {msig.address()}: {json.dumps(holding_assets, indent=2)} ")

rent_payment_index = account_info["assets"][0]["asset-id"]
print("rent_payment_index:", rent_payment_index)

created_assets = account_info["created-assets"]
print(f"created assets of {msig.address()}: {json.dumps(created_assets, indent=2)}")



# optin rent account for rents payment
sp = algod_client.suggested_params()
txn = AssetTransferTxn(
    sender=accounts[1],
    sp=params,
    receiver=accounts[1],
    amt=0,
    index=rent_payment_index,
    )

stxn = txn.sign(accountkey2)

try:
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4)  
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))   
except Exception as err:
    print(err)
    


# optin rent account for rents payment
sp = algod_client.suggested_params()
txn = AssetTransferTxn(
    sender=accounts[2],
    sp=params,
    receiver=accounts[2],
    amt=0,
    index=rent_payment_index,
    )

stxn = txn.sign(accountkey3)

try:
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4)  
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))   
except Exception as err:
    print(err)
    


# transfer rents assets to account2
version = 1  # multisig version
threshold = 2  # how many signatures are necessary
msig = Multisig(version, threshold, [accounts[0], accounts[1], accounts[2]])
msig.validate()

# get node suggested parameters
params = algod_client.suggested_params()
# comment out the next two (2) lines to use suggested fees
# params.flat_fee = True
# params.fee = 1000

txn = AssetTransferTxn(
    sender=msig.address(),
    sp=params,
    receiver=accounts[2],
    amt=100,
    index=rent_payment_index,
    )

mtx = MultisigTransaction(txn, msig)
mtx.sign(accountkey1)
mtx.sign(accountkey2)
# mtx.sign(accountkey3)

# print(json.dumps(mtx.multisig.json_dictify(), indent=2))
# print(mtx.transaction.dictify())
# print("Encoded transaction:", encoding.msgpack_encode(mtx))


try:
# send the transaction
    txid = algod_client.send_raw_transaction(encoding.msgpack_encode(mtx))    
    print("TXID: ", txid)   
    confirmed_txn = wait_for_confirmation(algod_client, txid, 6)  
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))
    print("Transaction information: {}".format(
        json.dumps(confirmed_txn, indent=4)))
    print("Decoded note: {}".format(base64.b64decode(
        confirmed_txn["txn"]["txn"]["note"]).decode()))
except Exception as err:
    print(err)    
    

# mint test tokens for lease
sp = algod_client.suggested_params()
txn = AssetConfigTxn(
    sender=accounts[1],
    sp=sp,
    total=1,
    default_frozen=False,
    unit_name="MART",
    asset_name="MEET_ART",
    manager=accounts[1],
    reserve=accounts[1],
    freeze=accounts[1],
    clawback=accounts[1],
    url="https://path/to/my/asset/art1/details", 
    decimals=0)

stxn = txn.sign(accountkey2)

try:
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4)  
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))   
except Exception as err:
    print(err)
    
account_info = algod_client.account_info(accounts[1])
print("balance of {}: {} microAlgos".format(accounts[1], account_info["amount"]))

holding_assets = account_info["assets"]
print(f"holding assets of {accounts[1]}: {json.dumps(holding_assets, indent=2)} ")

nft1_index = account_info["assets"][1]["asset-id"]
print("nft1_index:", nft1_index)

created_assets = account_info["created-assets"]
print(f"created assets of {accounts[1]}: {json.dumps(created_assets, indent=2)}")



# # mint test tokens for lease
# sp = algod_client.suggested_params()
# txn = AssetConfigTxn(
#     sender=accounts[2],
#     sp=sp,
#     total=1,
#     default_frozen=False,
#     unit_name="WART",
#     asset_name="WITH_ART",
#     manager=accounts[2],
#     reserve=accounts[2],
#     freeze=accounts[2],
#     clawback=accounts[2],
#     url="https://path/to/my/asset/art2/details", 
#     decimals=0)

# stxn = txn.sign(accountkey3)

# try:
#     txid = algod_client.send_transaction(stxn)
#     print("Signed transaction with txID: {}".format(txid))
#     # Wait for the transaction to be confirmed
#     confirmed_txn = wait_for_confirmation(algod_client, txid, 4)  
#     print("TXID: ", txid)
#     print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))   
# except Exception as err:
#     print(err)
    
# account_info = algod_client.account_info(accounts[2])
# print("balance of {}: {} microAlgos".format(accounts[2], account_info["amount"]))

# holding_assets = account_info["assets"]
# print(f"holding assets of {accounts[2]}: {json.dumps(holding_assets, indent=2)} ")

# nft2_index = account_info["assets"][0]["asset-id"]
# print("nft2_index:", nft2_index)

# created_assets = account_info["created-assets"]
# print(f"created assets of {accounts[2]}: {json.dumps(created_assets, indent=2)}")

TXID:  GFC27URPBJQLQ24JZXHGHTZ54A3W6DPV6X2LSG5YB6JPJUY2QDMA
Result confirmed in round: 30
Transaction information: {
    "asset-index": 5,
    "confirmed-round": 30,
    "pool-error": "",
    "txn": {
        "msig": {
            "subsig": [
                {
                    "pk": "QLbSxCBNwWpNDI0sTt4lGPOT313uTF/Nyt+Kulsk8j8=",
                    "s": "jWp9sDyluX8bAQTdOSxzdLp11LpYCGuJSn0H6oDy4ZFYqp7ATGHg5D36fZjx6vHs9Si8kfMszC+m6wAMR241AA=="
                },
                {
                    "pk": "cEAODiKMbSNoFJJvfHpPF6HKuZtoLd6UWGfFaaZBoc0=",
                    "s": "eeEG/RK1Ulp8girxjwvn8JxvhJLcOTFeF0Rx1EIyO0t5p9cUKPQxsnqSW2hYJf24yYJO7yqiBg/ZnJQ0gR2BCw=="
                },
                {
                    "pk": "cIuaU9QSJ9X8aLoh5MbJEI3N+VXbZXFf4ZEaxREttfM="
                }
            ],
            "thr": 2,
            "v": 1
        },
        "txn": {
            "apar": {
                "an": "metaone",
                "au": "https://path/to/my/asset/details

## deploy metaone smart contract
1. deploy the contract
2. set the initial admin at the same time

In [9]:
# deploy metaone market contract 

from contracts.metaone.market import approval,clear
from pyteal import compileTeal, Mode


# define private keys
creator_private_key = accountkey1

# declare application state storage (immutable)
local_ints = 9 + 1
local_bytes = 1
global_ints = 4
global_bytes = 1
global_schema = transaction.StateSchema(global_ints, global_bytes)
local_schema = transaction.StateSchema(local_ints, local_bytes)

# get PyTeal approval program
approval_program_ast = approval()
# compile program to TEAL assembly
approval_program_teal = compileTeal(
    approval_program_ast, mode=Mode.Application, version=6
)
# compile program to binary
approval_program_compiled = compile_program(algod_client, approval_program_teal)

# get PyTeal clear state program
clear_state_program_ast = clear()
# compile program to TEAL assembly
clear_state_program_teal = compileTeal(
    clear_state_program_ast, mode=Mode.Application, version=6
)
# compile program to binary
clear_state_program_compiled = compile_program(
    algod_client, clear_state_program_teal
)


# create list of bytes for app args
app_args = [b"set_admin"]

# create new application
app_id = create_app(
    algod_client,
    creator_private_key,
    approval_program_compiled,
    clear_state_program_compiled,
    global_schema,
    local_schema,
    app_args,
    accounts=[accounts[1]],
)

# read global state of application
global_state=read_global_state(
        algod_client, account.address_from_private_key(creator_private_key), app_id
    )

metaone_app_id = global_state['id']
print("metaone_app_id:",metaone_app_id)

# metaone_app_id = 53

Created new app-id: 10
metaone_app_id: 10


In [10]:
# can only run once
call_app(algod_client, accountkey2, metaone_app_id, [b"set_admin"], msig.address())

Call from account:  OBAA4DRCRRWSG2AUSJXXY6SPC6Q4VOM3NAW55FCYM7CWTJSBUHG67DNSVU
TX: {'sender': 'OBAA4DRCRRWSG2AUSJXXY6SPC6Q4VOM3NAW55FCYM7CWTJSBUHG67DNSVU', 'fee': 1000, 'first_valid_round': 40, 'last_valid_round': 1040, 'note': None, 'genesis_id': 'sandnet-v1', 'genesis_hash': 'WhNxQp0oM+XnShfmGyVTIm8yZmpzExyEA0zWyMDWkk0=', 'group': None, 'lease': None, 'type': 'appl', 'rekey_to': None, 'index': 10, 'on_complete': <OnComplete.NoOpOC: 0>, 'local_schema': None, 'global_schema': None, 'approval_program': None, 'clear_program': None, 'app_args': [b'set_admin'], 'accounts': ['6Y23LITYLQXVYWBHKZQEVG5A2ISMMEDFC6VMXXUPHN7F6P6NS76R5TF474'], 'foreign_apps': None, 'foreign_assets': None, 'extra_pages': 0}
TXID:  GTDMIUHAIH2RBNIPWIIHCPL55HCTY2SUFZWMQC6RT4KVV4IU7XDA
Result confirmed in round: 42
Called app-id:  10
Global State updated :

[
  {
    "key": "cGxhdF9hZG1pbg==",
    "value": {
      "action": 1,
      "bytes": "9jW1onhcL1xYJ1ZgSpug0iTGEGUXqsvejzt+Xz/Nl/0="
    }
  }
]


In [11]:
version = 1  # multisig version
threshold = 2  # how many signatures are necessary
msig = Multisig(version, threshold, [accounts[0], accounts[1], accounts[2]])
msig.validate()

# get node suggested parameters
params = algod_client.suggested_params()
# comment out the next two (2) lines to use suggested fees
params.flat_fee = True
params.fee = 1000

# create unsigned transaction
txn = ApplicationNoOpTxn(msig.address(), params, metaone_app_id, app_args=[b"set_admin"], accounts=[accounts[1]])
mtx = MultisigTransaction(txn, msig)

# sign transaction
mtx.sign(accountkey1)
mtx.sign(accountkey2)


try:
# send the transaction
    txid = algod_client.send_raw_transaction(encoding.msgpack_encode(mtx))    
    print("TXID: ", txid)   
    confirmed_txn = wait_for_confirmation(algod_client, txid, 6)  
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))
    print("Transaction information: {}".format(
        json.dumps(confirmed_txn, indent=4)))
    print("Decoded note: {}".format(base64.b64decode(
        confirmed_txn["txn"]["txn"]["note"]).decode()))
except Exception as err:
    print(err)



TXID:  5DXPFAZWJFM3EH2XQCJKO3W5KMGJ3JUR2X7DL227LNM3CY32M25Q
Result confirmed in round: 44
Transaction information: {
    "confirmed-round": 44,
    "global-state-delta": [
        {
            "key": "cGxhdF9hZG1pbg==",
            "value": {
                "action": 1,
                "bytes": "cEAODiKMbSNoFJJvfHpPF6HKuZtoLd6UWGfFaaZBoc0="
            }
        }
    ],
    "pool-error": "",
    "sender-rewards": 99,
    "txn": {
        "msig": {
            "subsig": [
                {
                    "pk": "QLbSxCBNwWpNDI0sTt4lGPOT313uTF/Nyt+Kulsk8j8=",
                    "s": "LWsYUnG+pX/CrrSX0TgSv7Xmh6gTILQK6+2Zr7mUXA0UmSP3LaGVwJ/iwkV93Y4+iJKEPUWbULX3ofsXQtmZCA=="
                },
                {
                    "pk": "cEAODiKMbSNoFJJvfHpPF6HKuZtoLd6UWGfFaaZBoc0=",
                    "s": "vMzJ4w/vpu4kWgoOp3HzDqDcgIPu3eR78oKIznhj7uJP/V/X4oeLt/l1GIrDPOCSNZBp/NtozK3MUxgiK3lvDg=="
                },
                {
                    "pk": "cIuaU9QSJ9X8aLoh5MbJEI

In [12]:
# can only run once
call_app(algod_client, accountkey2, metaone_app_id, [b"set_admin"], msig.address())

Call from account:  OBAA4DRCRRWSG2AUSJXXY6SPC6Q4VOM3NAW55FCYM7CWTJSBUHG67DNSVU
TX: {'sender': 'OBAA4DRCRRWSG2AUSJXXY6SPC6Q4VOM3NAW55FCYM7CWTJSBUHG67DNSVU', 'fee': 1000, 'first_valid_round': 44, 'last_valid_round': 1044, 'note': None, 'genesis_id': 'sandnet-v1', 'genesis_hash': 'WhNxQp0oM+XnShfmGyVTIm8yZmpzExyEA0zWyMDWkk0=', 'group': None, 'lease': None, 'type': 'appl', 'rekey_to': None, 'index': 10, 'on_complete': <OnComplete.NoOpOC: 0>, 'local_schema': None, 'global_schema': None, 'approval_program': None, 'clear_program': None, 'app_args': [b'set_admin'], 'accounts': ['6Y23LITYLQXVYWBHKZQEVG5A2ISMMEDFC6VMXXUPHN7F6P6NS76R5TF474'], 'foreign_apps': None, 'foreign_assets': None, 'extra_pages': 0}
TXID:  RQJVRRVZL3SEDQZWUFCOM4VFACAGV53LJLDFZTBV5UB5MTQVL4XA
Result confirmed in round: 46
Called app-id:  10
Global State updated :

[
  {
    "key": "cGxhdF9hZG1pbg==",
    "value": {
      "action": 1,
      "bytes": "9jW1onhcL1xYJ1ZgSpug0iTGEGUXqsvejzt+Xz/Nl/0="
    }
  }
]


In [13]:

# global_state = read_global_state(algod_client, app_creator, metaone_app_id)

# key = base64.b64decode("cGxhdF9hZG1pbg==".encode(encoding = 'UTF-8'))
# encoded_value = global_state['params']['global-state'][0]['value']['bytes']
# raw_bytes = base64.b64decode(encoded_value)
# address = encode_address(raw_bytes)
# print(f'{key}: {address}')

# for i in range(1,4):
#     encoded_key = (global_state['params']['global-state'][i]['key'])
#     key = base64.b64decode(encoded_key)
#     # key = base64.b64decode(key_raw.encode(encoding = 'UTF-8'))
#     value = global_state['params']['global-state'][i]['value']['uint']
#     print(f'{key}: {value}')

In [14]:
def opt_in_app(client, private_key, index):
    # declare sender
    sender = account.address_from_private_key(private_key)
    print("OptIn from account: ", sender)

    # get node suggested parameters
    params = client.suggested_params()
    # comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 1000

    # create unsigned transaction
    txn = transaction.ApplicationOptInTxn(sender, params, index)

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # await confirmation
    wait_for_confirmation(client, tx_id)

    # display results
    transaction_response = client.pending_transaction_info(tx_id)
    print("OptIn to app-id:", transaction_response["txn"]["txn"]["apid"])

In [15]:
opt_in_app(algod_client, accountkey2, metaone_app_id)

OptIn from account:  OBAA4DRCRRWSG2AUSJXXY6SPC6Q4VOM3NAW55FCYM7CWTJSBUHG67DNSVU
OptIn to app-id: 10


In [16]:
opt_in_app(algod_client, accountkey3, metaone_app_id)

OptIn from account:  OCFZUU6UCIT5L7DIXIQ6JRWJCCG436KV3NSXCX7BSENMKEJNWXZ5AF44BA
OptIn to app-id: 10


In [17]:

version = 1  # multisig version
threshold = 2  # how many signatures are necessary
msig = Multisig(version, threshold, [accounts[0], accounts[1], accounts[2]])
msig.validate()

# get node suggested parameters
params = algod_client.suggested_params()
# comment out the next two (2) lines to use suggested fees
params.flat_fee = True
params.fee = 1000

# create unsigned transaction
txn = transaction.ApplicationOptInTxn(msig.address(), params, metaone_app_id)
mtx = MultisigTransaction(txn, msig)

# sign transaction
mtx.sign(accountkey1)
mtx.sign(accountkey2)


try:
# send the transaction
    txid = algod_client.send_raw_transaction(encoding.msgpack_encode(mtx))    
    print("TXID: ", txid)   
    confirmed_txn = wait_for_confirmation(algod_client, txid, 6)  
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))
    print("Transaction information: {}".format(
        json.dumps(confirmed_txn, indent=4)))
    print("Decoded note: {}".format(base64.b64decode(
        confirmed_txn["txn"]["txn"]["note"]).decode()))
except Exception as err:
    print(err)


TXID:  4T4FKSADTFSQIDAH7BXRJGVDORU6D3342SVEUGNJMRTTAQJZ7VYA
Result confirmed in round: 52
Transaction information: {
    "confirmed-round": 52,
    "local-state-delta": [
        {
            "address": "6Y23LITYLQXVYWBHKZQEVG5A2ISMMEDFC6VMXXUPHN7F6P6NS76R5TF474",
            "delta": [
                {
                    "key": "aXRlbV9pbmRleA==",
                    "value": {
                        "action": 2
                    }
                },
                {
                    "key": "bWluX3RpbWVfdW5pdHM=",
                    "value": {
                        "action": 2
                    }
                },
                {
                    "key": "dGltZV91bml0",
                    "value": {
                        "action": 2
                    }
                },
                {
                    "key": "dXNlcl9hY2NvdW50",
                    "value": {
                        "action": 1
                    }
                },
                {
 

In [18]:
# app address 
def app_addr(app_id):
    addr = enc.encode_address(enc.checksum(b"appID" + (app_id).to_bytes(8, "big")))
    return addr

In [19]:
pay_start_fund(algod_client, sponsor, sponsorkey, app_addr(metaone_app_id), 200_000)

TX: {'sender': 'KI5HOKIYGKEOZ6VLNHKJGF4KXYNBZHXB7HSX5BQ2LYNB2IJLWFT7DIV2XI', 'fee': 1000, 'first_valid_round': 52, 'last_valid_round': 1052, 'note': None, 'genesis_id': 'sandnet-v1', 'genesis_hash': 'WhNxQp0oM+XnShfmGyVTIm8yZmpzExyEA0zWyMDWkk0=', 'group': None, 'lease': None, 'type': 'pay', 'rekey_to': None, 'receiver': 'Y5NP7DMHLJWMKUNZGKGG62TLLZEYE4AUYI7UV3NE7N2DNVK7WHD6RZW5WQ', 'amt': 200000, 'close_remainder_to': None}
TXID:  JHPB63EAOEPZFTF6NFH4B3RK67BLVCOU4T367PIIQIC2XRJ5XONQ
Result confirmed in round: 54


In [20]:
#set fee and make the contract optin the assets for rent payment

version = 1  # multisig version
threshold = 2  # how many signatures are necessary
msig = Multisig(version, threshold, [accounts[0], accounts[1], accounts[2]])
msig.validate()

# get node suggested parameters
params = algod_client.suggested_params()
# comment out the next two (2) lines to use suggested fees
params.flat_fee = True
params.fee = 2000

app_args = [b"set_fee", rent_payment_index, 10, 1]
foreign_assets = [(rent_payment_index)]

print ('foreign_assets len: ',len(foreign_assets))
# create unsigned transaction
txn = ApplicationNoOpTxn(msig.address(), params, metaone_app_id, app_args, foreign_assets=[rent_payment_index])
mtx = MultisigTransaction(txn, msig)


# multi_private_key_1 = mnemonic.to_private_key(multi_mnemonic1)
mtx.sign(accountkey1)
# multi_private_key_2 = mnemonic.to_private_key(multi_mnemonic2)
mtx.sign(accountkey2)

try:
# send the transaction
    txid = algod_client.send_raw_transaction(encoding.msgpack_encode(mtx))    
    print("TXID: ", txid)   
    confirmed_txn = wait_for_confirmation(algod_client, txid, 6)  
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))
    print("Transaction information: {}".format(
        json.dumps(confirmed_txn, indent=4)))
    print("Decoded note: {}".format(base64.b64decode(
        confirmed_txn["txn"]["txn"]["note"]).decode()))
except Exception as err:
    print(err)



foreign_assets len:  1
TXID:  B6SM45IXYXSG4YD6Y6GTSHZERIZ6CNRTDQGLIEEL4AHD52HDMGSA
Result confirmed in round: 56
Transaction information: {
    "confirmed-round": 56,
    "global-state-delta": [
        {
            "key": "Y3VycmVuY3k=",
            "value": {
                "action": 2,
                "uint": 5
            }
        },
        {
            "key": "c2V0X3JhdGVfZGVub21pbmF0b3I=",
            "value": {
                "action": 2,
                "uint": 10
            }
        },
        {
            "key": "c2V0X3JhdGVfbnVtZXJhdG9y",
            "value": {
                "action": 2,
                "uint": 1
            }
        }
    ],
    "inner-txns": [
        {
            "pool-error": "",
            "txn": {
                "txn": {
                    "arcv": "Y5NP7DMHLJWMKUNZGKGG62TLLZEYE4AUYI7UV3NE7N2DNVK7WHD6RZW5WQ",
                    "fv": 54,
                    "lv": 1054,
                    "snd": "Y5NP7DMHLJWMKUNZGKGG62TLLZEYE4AUYI7UV3NE

In [21]:

# get node suggested parameters
params = algod_client.suggested_params()
# comment out the next two (2) lines to use suggested fees
params.flat_fee = True
params.fee = 3000

app_args = [b"offer", nft1_index, 60*3600*24, 10, 1, 2, 1]
foreign_assets = [(nft1_index)]

print ('foreign_assets len: ',len(foreign_assets))
market_address = app_addr(metaone_app_id)
print('market addr: ', market_address)
# create unsigned transaction
txn0 = transaction.PaymentTxn(accounts[1], params, app_addr(metaone_app_id),100_000)
txn1 = ApplicationNoOpTxn(accounts[1], params, metaone_app_id, app_args, foreign_assets=[nft1_index])
txn2 = transaction.AssetTransferTxn(accounts[1], params, market_address , 1, nft1_index)

gid = transaction.calculate_group_id([txn0, txn1, txn2])
txn0.group = gid
txn1.group = gid
txn2.group = gid

# sign transactions
stxn0 = txn0.sign(accountkey2)
stxn1 = txn1.sign(accountkey2)
stxn2 = txn2.sign(accountkey2)

try:
    txid = algod_client.send_transactions([stxn0, stxn1, stxn2])
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4)  
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))   
except Exception as err:
    print(err)


foreign_assets len:  1
market addr:  Y5NP7DMHLJWMKUNZGKGG62TLLZEYE4AUYI7UV3NE7N2DNVK7WHD6RZW5WQ
Signed transaction with txID: VMXX2N7DVMPE3NIAZDWPAE3TRZUOYABICX2TSKN42NXPMOV6YW5A
TXID:  VMXX2N7DVMPE3NIAZDWPAE3TRZUOYABICX2TSKN42NXPMOV6YW5A
Result confirmed in round: 61


In [22]:
# get node suggested parameters
params = algod_client.suggested_params()
# comment out the next two (2) lines to use suggested fees
# params.flat_fee = True
params.fee = 2000

app_args = [b"offer", nft1_index, 10, 30, 1, 2, 1]
foreign_assets = [(nft1_index)]

txn = ApplicationNoOpTxn(accounts[1], params, metaone_app_id, app_args, foreign_assets=[nft1_index])
signed_txn = txn.sign(accountkey2)

tx_id = signed_txn.transaction.get_txid()

# send transaction
algod_client.send_transactions([signed_txn])

print(signed_txn)

# await confirmation

confirmed_txn = wait_for_confirmation(algod_client, tx_id, 4)
print("TX:",txn)
print("TXID: ", tx_id)
print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))



TX: {'sender': 'OBAA4DRCRRWSG2AUSJXXY6SPC6Q4VOM3NAW55FCYM7CWTJSBUHG67DNSVU', 'fee': 558000, 'first_valid_round': 79, 'last_valid_round': 1079, 'note': None, 'genesis_id': 'sandnet-v1', 'genesis_hash': 'WhNxQp0oM+XnShfmGyVTIm8yZmpzExyEA0zWyMDWkk0=', 'group': None, 'lease': None, 'type': 'appl', 'rekey_to': None, 'index': 10, 'on_complete': <OnComplete.NoOpOC: 0>, 'local_schema': None, 'global_schema': None, 'approval_program': None, 'clear_program': None, 'app_args': [b'offer', b'\x00\x00\x00\x00\x00\x00\x00\t', b'\x00\x00\x00\x00\x00\x00\x00\n', b'\x00\x00\x00\x00\x00\x00\x00\x1e', b'\x00\x00\x00\x00\x00\x00\x00\x01', b'\x00\x00\x00\x00\x00\x00\x00\x02', b'\x00\x00\x00\x00\x00\x00\x00\x01'], 'accounts': None, 'foreign_apps': None, 'foreign_assets': [9], 'extra_pages': 0}
TXID:  ZYIJ5UMH3WJJSKVIMDQXXY7A4BS4ZDHNSWBZBKRGIKGIWXDGPAWQ
Result confirmed in round: 81


In [23]:
 # get node suggested parameters
params = algod_client.suggested_params()
# comment out the next two (2) lines to use suggested fees
params.flat_fee = True
params.fee = 2000

app_args = [b"rent", 20]
foreign_assets = [(rent_payment_index)]

print ('foreign_assets len: ',len(foreign_assets))
market_address = app_addr(metaone_app_id)
print('market addr: ', market_address)
# create unsigned transaction
txn1 = ApplicationNoOpTxn(accounts[2], params, metaone_app_id, app_args, 
                          foreign_assets=[rent_payment_index],
                          accounts=[accounts[1]])
txn2 = transaction.AssetTransferTxn(accounts[2], params, market_address , 20, rent_payment_index)

gid = transaction.calculate_group_id([txn1, txn2])
txn1.group = gid
txn2.group = gid

# sign transactions
stxn1 = txn1.sign(accountkey3)
stxn2 = txn2.sign(accountkey3)

try:
    txid = algod_client.send_transactions([stxn1, stxn2])
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4)  
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))   
except Exception as err:
    print(err)

foreign_assets len:  1
market addr:  Y5NP7DMHLJWMKUNZGKGG62TLLZEYE4AUYI7UV3NE7N2DNVK7WHD6RZW5WQ
Signed transaction with txID: 2NY2GRK6SWUOFMXA2HKZAIZRJB67QDAFQG3NW2QSAP6N5UC4FYRA
TXID:  2NY2GRK6SWUOFMXA2HKZAIZRJB67QDAFQG3NW2QSAP6N5UC4FYRA
Result confirmed in round: 101


In [26]:
# get node suggested parameters
params = algod_client.suggested_params()
# comment out the next two (2) lines to use suggested fees
# params.flat_fee = True
params.fee = 2000

app_args = [b"callback"]
foreign_assets = [(nft1_index)]

txn = ApplicationNoOpTxn(accounts[1], params, metaone_app_id, app_args, foreign_assets=[nft1_index])
signed_txn = txn.sign(accountkey2)

tx_id = signed_txn.transaction.get_txid()

# send transaction
algod_client.send_transactions([signed_txn])

print(signed_txn)

# await confirmation

confirmed_txn = wait_for_confirmation(algod_client, tx_id, 4)
print("TX:",txn)
print("TXID: ", tx_id)
print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))

TX: {'sender': 'OBAA4DRCRRWSG2AUSJXXY6SPC6Q4VOM3NAW55FCYM7CWTJSBUHG67DNSVU', 'fee': 446000, 'first_valid_round': 151, 'last_valid_round': 1151, 'note': None, 'genesis_id': 'sandnet-v1', 'genesis_hash': 'WhNxQp0oM+XnShfmGyVTIm8yZmpzExyEA0zWyMDWkk0=', 'group': None, 'lease': None, 'type': 'appl', 'rekey_to': None, 'index': 10, 'on_complete': <OnComplete.NoOpOC: 0>, 'local_schema': None, 'global_schema': None, 'approval_program': None, 'clear_program': None, 'app_args': [b'callback'], 'accounts': None, 'foreign_apps': None, 'foreign_assets': [9], 'extra_pages': 0}
TXID:  G3YZOSFORNQZ46Y4Z5MDOHFP2Z223QNP5AJ7HI2YWRHXM3P7EAQQ
Result confirmed in round: 153


In [24]:
# get node suggested parameters
params = algod_client.suggested_params()
# comment out the next two (2) lines to use suggested fees
# params.flat_fee = True
params.fee = 2000

app_args = [b"collect", 20]
foreign_assets = [(rent_payment_index)]

txn = ApplicationNoOpTxn(accounts[1], params, metaone_app_id, app_args, foreign_assets=[rent_payment_index])
signed_txn = txn.sign(accountkey2)

tx_id = signed_txn.transaction.get_txid()

# send transaction
algod_client.send_transactions([signed_txn])

print(signed_txn)

# await confirmation

confirmed_txn = wait_for_confirmation(algod_client, tx_id, 4)
print("TX:",txn)
print("TXID: ", tx_id)
print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))


TX: {'sender': 'OBAA4DRCRRWSG2AUSJXXY6SPC6Q4VOM3NAW55FCYM7CWTJSBUHG67DNSVU', 'fee': 462000, 'first_valid_round': 103, 'last_valid_round': 1103, 'note': None, 'genesis_id': 'sandnet-v1', 'genesis_hash': 'WhNxQp0oM+XnShfmGyVTIm8yZmpzExyEA0zWyMDWkk0=', 'group': None, 'lease': None, 'type': 'appl', 'rekey_to': None, 'index': 10, 'on_complete': <OnComplete.NoOpOC: 0>, 'local_schema': None, 'global_schema': None, 'approval_program': None, 'clear_program': None, 'app_args': [b'collect', b'\x00\x00\x00\x00\x00\x00\x00\x14'], 'accounts': None, 'foreign_apps': None, 'foreign_assets': [5], 'extra_pages': 0}
TXID:  UJNR7OV5E2DV5F7AELYDUNWUK3SDFD736TU6FUXWSQ2MEJB4C5DQ
Result confirmed in round: 105


In [26]:
# get node suggested parameters
params = algod_client.suggested_params()
# comment out the next two (2) lines to use suggested fees
# params.flat_fee = True
params.fee = 2000


txn = ApplicationCloseOutTxn(accounts[1], params, metaone_app_id)
signed_txn = txn.sign(accountkey2)

tx_id = signed_txn.transaction.get_txid()

# send transaction
algod_client.send_transactions([signed_txn])

print(signed_txn)

# await confirmation

confirmed_txn = wait_for_confirmation(algod_client, tx_id, 4)
print("TX:",txn)
print("TXID: ", tx_id)
print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))
# display results
transaction_response = algod_client.pending_transaction_info(tx_id)
print("Called app-id: ",transaction_response['txn']['txn']['apid'])
if "global-state-delta" in transaction_response :
    print("Global State updated :\n")
    print(json.dumps(transaction_response['global-state-delta'], indent=2))
if "local-state-delta" in transaction_response :
    print("Local State updated :\n")
    print(json.dumps(transaction_response['local-state-delta'], indent=2))

TX: {'sender': 'TXJETRP5AQNZFKCF5WUT3SA3727JNQI2VGUVZ5BMI2THPFNYUE5ACAC73U', 'fee': 414000, 'first_valid_round': 271, 'last_valid_round': 1271, 'note': None, 'genesis_id': 'sandnet-v1', 'genesis_hash': 'WhNxQp0oM+XnShfmGyVTIm8yZmpzExyEA0zWyMDWkk0=', 'group': None, 'lease': None, 'type': 'appl', 'rekey_to': None, 'index': 10, 'on_complete': <OnComplete.CloseOutOC: 2>, 'local_schema': None, 'global_schema': None, 'approval_program': None, 'clear_program': None, 'app_args': None, 'accounts': None, 'foreign_apps': None, 'foreign_assets': None, 'extra_pages': 0}
TXID:  FQACGEVI4ZDCK25I6AILTUKOKOG6QBYQRBDSWSGGZUFNJSISJVCA
Result confirmed in round: 273
Called app-id:  10


In [25]:
# get node suggested parameters
params = algod_client.suggested_params()
# comment out the next two (2) lines to use suggested fees
# params.flat_fee = True
params.fee = 2000

app_args = [b"collect", 20]
foreign_assets = [(rent_payment_index)]

txn = ApplicationNoOpTxn(accounts[1], params, metaone_app_id, app_args, foreign_assets=[rent_payment_index])
signed_txn = txn.sign(accountkey2)

tx_id = signed_txn.transaction.get_txid()

# send transaction
algod_client.send_transactions([signed_txn])

print(signed_txn)

# await confirmation

confirmed_txn = wait_for_confirmation(algod_client, tx_id, 4)
print("TX:",txn)
print("TXID: ", tx_id)
print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))



AlgodHTTPError: TransactionPool.Remember: transaction YIIXHZMJ2HCVN4BA5GT55M3Z6LDVNIDGPZV6RAAQPXTF3HVWR7MQ: logic eval error: assert failed pc=1596. Details: pc=1596, opcodes=load 20
>=
assert


In [27]:
# get node suggested parameters
params = algod_client.suggested_params()
# comment out the next two (2) lines to use suggested fees
# params.flat_fee = True
params.fee = 2000


txn = ApplicationCloseOutTxn(accounts[2], params, metaone_app_id)
signed_txn = txn.sign(accountkey3)

tx_id = signed_txn.transaction.get_txid()

# send transaction
algod_client.send_transactions([signed_txn])

print(signed_txn)

# await confirmation

confirmed_txn = wait_for_confirmation(algod_client, tx_id, 4)
print("TX:",txn)
print("TXID: ", tx_id)
print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))
# display results
transaction_response = algod_client.pending_transaction_info(tx_id)
print("Called app-id: ",transaction_response['txn']['txn']['apid'])
if "global-state-delta" in transaction_response :
    print("Global State updated :\n")
    print(json.dumps(transaction_response['global-state-delta'], indent=2))
if "local-state-delta" in transaction_response :
    print("Local State updated :\n")
    print(json.dumps(transaction_response['local-state-delta'], indent=2))

TX: {'sender': 'XDSMNP7T37R2ECDBL5LIZC2H5ON7LIQ23RJG5DUFNXQITUYXMTOCLWM7JQ', 'fee': 414000, 'first_valid_round': 273, 'last_valid_round': 1273, 'note': None, 'genesis_id': 'sandnet-v1', 'genesis_hash': 'WhNxQp0oM+XnShfmGyVTIm8yZmpzExyEA0zWyMDWkk0=', 'group': None, 'lease': None, 'type': 'appl', 'rekey_to': None, 'index': 10, 'on_complete': <OnComplete.CloseOutOC: 2>, 'local_schema': None, 'global_schema': None, 'approval_program': None, 'clear_program': None, 'app_args': None, 'accounts': None, 'foreign_apps': None, 'foreign_assets': None, 'extra_pages': 0}
TXID:  DT3QSG7HFZFW2M35ZJUPES3RZIPUXGVS3DICRAOYASKT6FACU2NQ
Result confirmed in round: 275
Called app-id:  10


In [ ]:

# deploy counter contract 

from contracts.counter.step_02 import approval,clear
from pyteal import compileTeal, Mode


# define private keys
creator_private_key = accountkey1

# declare application state storage (immutable)
local_ints = 0
local_bytes = 0
global_ints = 1
global_bytes = 1
global_schema = transaction.StateSchema(global_ints, global_bytes)
local_schema = transaction.StateSchema(local_ints, local_bytes)

# get PyTeal approval program
approval_program_ast = approval()
# compile program to TEAL assembly
approval_program_teal = compileTeal(
    approval_program_ast, mode=Mode.Application, version=5
)
# compile program to binary
approval_program_compiled = compile_program(algod_client, approval_program_teal)

# get PyTeal clear state program
clear_state_program_ast = clear()
# compile program to TEAL assembly
clear_state_program_teal = compileTeal(
    clear_state_program_ast, mode=Mode.Application, version=5
)
# compile program to binary
clear_state_program_compiled = compile_program(
    algod_client, clear_state_program_teal
)


# create list of bytes for app args
app_args = []

# create new application
app_id = create_app(
    algod_client,
    creator_private_key,
    approval_program_compiled,
    clear_state_program_compiled,
    global_schema,
    local_schema,
    app_args,
)

# read global state of application
global_state=read_global_state(
        algod_client, account.address_from_private_key(creator_private_key), app_id
    )

counter_app_id = global_state['id']
print("app_id:",counter_app_id)



call_app(algod_client, accountkey1, counter_app_id, [b"inc"], msig.address())
call_app(algod_client, accountkey1, counter_app_id, [b"dec"])


version = 1  # multisig version
threshold = 2  # how many signatures are necessary
msig = Multisig(version, threshold, [accounts[0], accounts[1], accounts[2]])
msig.validate()

# get node suggested parameters
params = algod_client.suggested_params()
# comment out the next two (2) lines to use suggested fees
# params.flat_fee = True
# params.fee = 1000

app_args = [b"inc"]
# create unsigned transaction
txn = ApplicationNoOpTxn(msig.address(), params, counter_app_id, app_args)


mtx = MultisigTransaction(txn, msig)
mtx.sign(accountkey1)
mtx.sign(accountkey2)
# mtx.sign(accountkey3)

# print(json.dumps(mtx.multisig.json_dictify(), indent=2))
# print(mtx.transaction.dictify())
# print("Encoded transaction:", encoding.msgpack_encode(mtx))


try:
# send the transaction
    txid = algod_client.send_raw_transaction(encoding.msgpack_encode(mtx))    
    print("TXID: ", txid)   
    confirmed_txn = wait_for_confirmation(algod_client, txid, 6)  
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))
    print("Transaction information: {}".format(
        json.dumps(confirmed_txn, indent=4)))
    print("Decoded note: {}".format(base64.b64decode(
        confirmed_txn["txn"]["txn"]["note"]).decode()))
except Exception as err:
    print(err)

